# M2608.001300 기계학습 기초 및 전기정보 응용<br> Assignment #4 Part 1: Hidden Markov Model (HMM)

Copyright (C) Data Science & AI Laboratory, Seoul National University. This material is for educational uses only. Some contents are based on the material provided by other paper/book authors and may be copyrighted by them. Written by Dongjin Lee, May 2022.

**For understanding of this work, please carefully look at given PPT file.**

Now, you're going to leave behind your implementations and instead migrate to one of popular deep learning frameworks, **PyTorch**. <br>
In this notebook, you will learn how to train a Hidden Markov Model (HMM) for sequential modeling. <br>
You need to follow the instructions to **complete 5 TODO sections and explain them if needed.**

You will see:
- what HMM is and when you might want to use it;
- the so-called "three problems" of an HMM; and 
- how to implement HMM in PyTorch.


**Note**: certain details are missing or ambiguous on purpose, in order to test your knowledge on the related materials. However, if you really feel that something essential is missing and cannot proceed to the next step, then contact the teaching staff with clear description of your problem.

<font color=red>**DO NOT clear the final outputs**</font> so that TAs can grade both your code and results.


### Some helpful tutorials and references for assignment #4:
- [1] Pytorch official documentation. [[link]](https://pytorch.org/docs/stable/index.html)
- [2] Stanford CS231n lectures. [[link]](http://cs231n.stanford.edu/)
- [3] https://ratsgo.github.io/machine%20learning/2017/03/18/HMMs/
- [4] https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/

## Problem 1. Implementing Hidden Markov Model

A hypothetical scenario
------

To motivate the use of HMMs, imagine that you have a friend who gets to do a lot of travelling. Every day, this jet-setting friend sends you a selfie from the city they’re in, to make you envious.

<center>

![Diagram of a traveling friend sending selfies](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/selfies.png?raw=true)
</center>





How would you go about guessing which city the friend is in each day, just by looking at the selfies?

If the selfie contains a really obvious landmark, like the Eiffel Tower, it will be easy to figure out where the photo was taken. If not, it will be a lot harder to infer the city.

But we have a clue to help us: the city the friend is in each day is not totally random. For example, the friend will probably remain in the same city for a few days to sightsee before flying to a new city.

## The HMM setup

The hypothetical scenario of the friend travelling between cities and sending you selfies can be modeled using an HMM.


An HMM models a system that is in a particular state at any given time and produces an output that depends on that state. 

At each timestep or clock tick, the system randomly decides on a new state and jumps into that state. The system then randomly generates an observation. The states are "hidden": we can't observe them. (In the cities/selfies analogy, the unknown cities would be the hidden states, and the selfies would be the observations.)

Let's denote the sequence of states as $\mathbf{z} = \{z_1, z_2, \dots, z_T \}$, where each state is one of a finite set of $N$ states, and the sequence of observations as $\mathbf{x} = \{x_1, x_2, \dots, x_T\}$. The observations could be discrete, like letters, or real-valued, like audio frames.

<center>

![Diagram of an HMM for three timesteps](https://github.com/lorenlugosch/pytorch_HMM/blob/master/img/hmm.png?raw=true)
</center>

An HMM makes two key assumptions:
- **Assumption 1:** The state at time $t$ depends *only* on the state at the previous time $t-1$. 
- **Assumption 2:** The output at time $t$ depends *only* on the state at time $t$.

These two assumptions make it possible to efficiently compute certain quantities that we may be interested in.

## Components of an HMM
An HMM has three sets of trainable parameters.
  


- The **transition model** is a square matrix $A$, where $A_{s, s'}$ represents $p(z_t = s|z_{t-1} = s')$, the probability of jumping from state $s'$ to state $s$. 

- The **emission model** $b_s(x_t)$ tells us $p(x_t|z_t = s)$, the probability of generating $x_t$ when the system is in state $s$. For discrete observations, which we will use in this notebook, the emission model is just a lookup table, with one row for each state, and one column for each observation. For real-valued observations, it is common to use a Gaussian mixture model or neural network to implement the emission model. 

- The **state priors** tell us $p(z_1 = s)$, the probability of starting in state $s$. We use $\pi$ to denote the vector of state priors, so $\pi_s$ is the state prior for state $s$.

Let's program an HMM class in PyTorch.

In [1]:
import torch
import numpy as np

class HMM(torch.nn.Module):
  """
  Hidden Markov Model with discrete observations.
  """
  def __init__(self, M, N):
    super(HMM, self).__init__()
    self.M = M # number of possible observations
    self.N = N # number of states

    # A
    self.transition_model = TransitionModel(self.N)

    # b(x_t)
    self.emission_model = EmissionModel(self.N,self.M)

    # pi
    self.unnormalized_state_priors = torch.nn.Parameter(torch.randn(self.N))

    # use the GPU
    self.is_cuda = torch.cuda.is_available()
    if self.is_cuda: self.cuda()

class TransitionModel(torch.nn.Module):
  def __init__(self, N):
    super(TransitionModel, self).__init__()
    self.N = N
    self.unnormalized_transition_matrix = torch.nn.Parameter(torch.randn(N,N))

class EmissionModel(torch.nn.Module):
  def __init__(self, N, M):
    super(EmissionModel, self).__init__()
    self.N = N
    self.M = M
    self.unnormalized_emission_matrix = torch.nn.Parameter(torch.randn(N,M))

To sample from the HMM, we start by picking a random initial state from the state prior distribution.

Then, we sample an output from the emission distribution, sample a transition from the transition distribution, and repeat.

(Notice that we pass the unnormalized model parameters through a softmax function to make them into probabilities.)


In [2]:
def sample(self, T=10):
  state_priors = torch.nn.functional.softmax(self.unnormalized_state_priors, dim=0)
  transition_matrix = torch.nn.functional.softmax(self.transition_model.unnormalized_transition_matrix, dim=0)
  emission_matrix = torch.nn.functional.softmax(self.emission_model.unnormalized_emission_matrix, dim=1)

  # sample initial state
  z_t = torch.distributions.categorical.Categorical(state_priors).sample().item()
  z = []; x = []
  z.append(z_t)
  for t in range(0,T):
    # sample emission
    x_t = torch.distributions.categorical.Categorical(emission_matrix[z_t]).sample().item()
    x.append(x_t)

    # sample transition
    z_t = torch.distributions.categorical.Categorical(transition_matrix[:,z_t]).sample().item()
    if t < T-1: z.append(z_t)

  return x, z

# Add the sampling method to our HMM class
HMM.sample = sample

Let's try hard-coding an HMM for generating fake words. (We'll also add some helper functions for encoding and decoding strings.)

We will assume that the system has one state for generating vowels and one state for generating consonants, and the transition matrix has 0s on the diagonal---in other words, the system cannot stay in the vowel state or the consonant state for more than one timestep; it has to switch.

Since we pass the transition matrix through a softmax, to get 0s we set the unnormalized parameter values to $-\infty$.

In [3]:
import string
alphabet = string.ascii_lowercase

def encode(s):
  """
  Convert a string into a list of integers
  """
  x = [alphabet.index(ss) for ss in s]
  return x

def decode(x):
  """
  Convert list of ints to string
  """
  s = "".join([alphabet[xx] for xx in x])
  return s

# Initialize the model
model = HMM(M=len(alphabet), N=2) 

# Hard-wiring the parameters!
# Let state 0 = consonant, state 1 = vowel
for p in model.parameters():
    p.requires_grad = False # needed to do lines below
model.unnormalized_state_priors[0] = 0.    # Let's start with a consonant more frequently
model.unnormalized_state_priors[1] = -0.5
print("State priors:", torch.nn.functional.softmax(model.unnormalized_state_priors, dim=0))

# In state 0, only allow consonants; in state 1, only allow vowels
vowel_indices = torch.tensor([alphabet.index(letter) for letter in "aeiou"])
consonant_indices = torch.tensor([alphabet.index(letter) for letter in "bcdfghjklmnpqrstvwxyz"])
model.emission_model.unnormalized_emission_matrix[0, vowel_indices] = -np.inf
model.emission_model.unnormalized_emission_matrix[1, consonant_indices] = -np.inf 
print("Emission matrix:", torch.nn.functional.softmax(model.emission_model.unnormalized_emission_matrix, dim=1))

# Only allow vowel -> consonant and consonant -> vowel
model.transition_model.unnormalized_transition_matrix[0,0] = -np.inf  # consonant -> consonant
model.transition_model.unnormalized_transition_matrix[0,1] = 0.       # vowel -> consonant
model.transition_model.unnormalized_transition_matrix[1,0] = 0.       # consonant -> vowel
model.transition_model.unnormalized_transition_matrix[1,1] = -np.inf  # vowel -> vowel
print("Transition matrix:", torch.nn.functional.softmax(model.transition_model.unnormalized_transition_matrix, dim=0))



State priors: tensor([0.6225, 0.3775], device='cuda:0')
Emission matrix: tensor([[0.0000, 0.0132, 0.0057, 0.0412, 0.0000, 0.0096, 0.0520, 0.0690, 0.0000,
         0.0203, 0.0206, 0.0586, 0.0449, 0.0074, 0.0000, 0.0226, 0.0362, 0.1019,
         0.0173, 0.2785, 0.0000, 0.0137, 0.1334, 0.0081, 0.0102, 0.0357],
        [0.1228, 0.0000, 0.0000, 0.0000, 0.0655, 0.0000, 0.0000, 0.0000, 0.5361,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1241, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.1516, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       device='cuda:0')
Transition matrix: tensor([[0., 1.],
        [1., 0.]], device='cuda:0')


**TODO 1: Try sampling from our hard-coded model.**


Sampled outputs should have the form of: <br>
```
x: adarituhij
z: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
```



In [4]:
# Sample at least 3 pairs of (x, z) outputs
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

def printXandZ (x, z):
  x = x[::-1]
  print("x:", end =" ")
  for xx in x:
    print(alphabet[xx], end ="")
  print()
  print("z:", z)
  print()
  return

for _ in range(0, 10):
  x, z = model.sample()
  printXandZ(x, z)
  


##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

x: eziwitilis
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: liderutoqo
z: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]

x: ohuwotigit
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: atutatutir
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: etiwitarit
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: itetadites
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: imatanihim
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: itiritotiz
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: itoyiwigih
z: [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]

x: tolidiridi
z: [1, 0, 1, 0, 1, 0, 1, 0, 1, 0]



## Problem 2. The Three Problems

The "three problems" that need to be solved before you can effectively use an HMM are:
- Problem 1: How do we efficiently compute $p(\mathbf{x})$?
- Problem 2: How do we find the most likely state sequence $\mathbf{z}$ that could have generated the data? 
- Problem 3: How do we train the model?

In the rest of the notebook, we will see how to solve each problem and implement the solutions in PyTorch.

### Problem 2-1. How do we compute $p(\mathbf{x})$?


#### *Why?*
Why might we care about computing $p(\mathbf{x})$? Here's two reasons.
* Given two HMMs, $\theta_1$ and $\theta_2$, we can compute the likelihood of some data $\mathbf{x}$ under each model, $p_{\theta_1}(\mathbf{x})$ and $p_{\theta_2}(\mathbf{x})$, to decide which model is a better fit to the data. 

  (For example, given an HMM for English speech and an HMM for French speech, we could compute the likelihood given each model, and pick the model with the higher likelihood to infer whether the person is speaking English or French.)
* Being able to compute $p(\mathbf{x})$ gives us a way to train the model, as we will see later.

#### *How?*
Given that we want $p(\mathbf{x})$, how do we compute it?

We've assumed that the data is generated by visiting some sequence of states $\mathbf{z}$ and picking an output $x_t$ for each $z_t$ from the emission distribution $p(x_t|z_t)$. So if we knew $\mathbf{z}$, then the probability of $\mathbf{x}$ could be computed as follows:

$$p(\mathbf{x}|\mathbf{z}) = \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

However, we don't know $\mathbf{z}$; it's hidden. But we do know the probability of any given $\mathbf{z}$, independent of what we observe. So we could get the probability of $\mathbf{x}$ by summing over the different possibilities for $\mathbf{z}$, like this:

$$p(\mathbf{x}) = \sum_{\mathbf{z}} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z}) = \sum_{\mathbf{z}} p(\mathbf{z}) \prod_{t} p(x_t|z_t) p(z_t|z_{t-1})$$

The problem is: if you try to take that sum directly, you will need to compute $N^T$ terms. This is impossible to do for anything but very short sequences. For example, let's say the sequence is of length $T=100$ and there are $N=2$ possible states. Then we would need to check $N^T = 2^{100} \approx 10^{30}$ different possible state sequences.

We need a way to compute $p(\mathbf{x})$ that doesn't require us to explicitly calculate all $N^T$ terms. For this, we use the forward algorithm.

________

<u><b>The Forward Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\alpha_{s,1} := b_s(x_1) \cdot \pi_s$ 
> 
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\alpha_{s,t} := b_s(x_t) \cdot \underset{s'}{\sum} A_{s, s'} \cdot \alpha_{s',t-1} $
> 
> $p(\mathbf{x}) := \underset{s}{\sum} \alpha_{s,T}$\
> return $p(\mathbf{x})$
________


The forward algorithm is much faster than enumerating all $N^T$ possible state sequences: it requires only $O(N^2T)$ operations to run, since each step is mostly multiplying the vector of forward variables by the transition matrix. (And very often we can reduce that complexity even further, if the transition matrix is sparse.)

There is one practical problem with the forward algorithm as presented above: it is prone to underflow due to multiplying a long chain of small numbers, since probabilities are always between 0 and 1. Instead, let's do everything in the log domain. In the log domain, a multiplication becomes a sum, and a sum becomes a ${\text{logsumexp}}$.

Let $x \in \mathbb{R}^n$, ${\text{logsumexp}} (x)$ is defined as:

$${\text{logsumexp}} (x) = \log (\sum_{i} \exp (x_i)).$$

Numerically, ${\text{logsumexp}}$ is similar to ${\max}$: in fact, it’s sometimes called the “smooth maximum” function. For example:

$$\max ([1, 2, 3]) = 3 $$
$${\text{logsumexp}} ([1, 2, 3]) = 3.4076$$

________

<u><b>The Forward Algorithm (Log Domain)</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\text{log }\alpha_{s,1} := \text{log }b_s(x_1) + \text{log }\pi_s$ 
> 
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
> $\text{log }\alpha_{s,t} := \text{log }b_s(x_t) +  \underset{s'}{\text{logsumexp}} \left( \text{log }A_{s, s'} + \text{log }\alpha_{s',t-1} \right)$
> 
> $\text{log }p(\mathbf{x}) := \underset{s}{\text{logsumexp}} \left( \text{log }\alpha_{s,T} \right)$\
> return $\text{log }p(\mathbf{x})$
________

**TODO 2**: Now that we have a numerically stable version of the forward algorithm, let's implement it in PyTorch. 

In [5]:
def HMM_forward(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)

  Compute log p(x) for each example in the batch.
  T = length of each example
  """
  ##############################################################################
  #                          IMPLEMENT YOUR CODE                               #
  ##############################################################################

  # initialization
  log_state_priors = torch.log(torch.nn.functional.softmax(self.unnormalized_state_priors, dim=0)) # (N)
  log_emission = self.emission_model.forward(x[:, 0]) # (batch size, N)
  log_alpha = log_state_priors.reshape((1, -1)) + log_emission  # (batch size, N)

  # induction
  for t in range(1, x.shape[1]):
    log_emission = self.emission_model.forward(x[:, t]) # (batch size, N)
    logsumexp = self.transition_model.forward(log_alpha) # (batch size, N)
    log_alpha = log_emission + logsumexp  # (batch size, N)

  # termination
  log_probs = torch.logsumexp(log_alpha, dim=1)

  ##############################################################################
  #                          END OF YOUR CODE                                  #
  ##############################################################################  
  return log_probs # (batch size)

def emission_model_forward(self, x_t):
  """
  x_t : IntTensor of shape (batch size)
  """
  ##############################################################################
  #                          IMPLEMENT YOUR CODE                               #
  ##############################################################################
  
  out = self.unnormalized_emission_matrix # (N, M)
  out = torch.nn.functional.softmax(out, dim=1) # (N, M)
  out = out.T[x_t, :] # (batch size, N)
  out = torch.log(out) # (batch size, N)

  ##############################################################################
  #                          END OF YOUR CODE                                  #
  ##############################################################################
  return out

def transition_model_forward(self, log_alpha):
  """
  log_alpha : Tensor of shape (batch size, N)
  Multiply previous timestep's alphas by transition matrix (in log domain)
  """
  ##############################################################################
  #                          IMPLEMENT YOUR CODE                               #
  ##############################################################################
  
  out = self.unnormalized_transition_matrix # (N, N)
  out = torch.nn.functional.softmax(out, dim=0)
  out = torch.log(out) # (N, N)
  out = log_domain_matmul(log_alpha, out) # (batch size, N)

  ##############################################################################
  #                          END OF YOUR CODE                                  #
  ##############################################################################
  return out

def log_domain_matmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Normally, a matrix multiplication
	computes out_{i,j} = sum_k A_{i,k} x B_{k,j}

	A log domain matrix multiplication
	computes out_{i,j} = logsumexp_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	# log_A_expanded = torch.stack([log_A] * p, dim=2)
	# log_B_expanded = torch.stack([log_B] * m, dim=0)
    # fix for PyTorch > 1.5 by egaznep on Github:
	log_A_expanded = torch.reshape(log_A, (m,n,1))
	log_B_expanded = torch.reshape(log_B, (1,n,p))

	elementwise_sum = log_A_expanded + log_B_expanded
	out = torch.logsumexp(elementwise_sum, dim=1)

	return out

TransitionModel.forward = transition_model_forward
EmissionModel.forward = emission_model_forward
HMM.forward = HMM_forward

Try running the forward algorithm on our vowels/consonants model from before:

In [6]:
x = torch.stack( [torch.tensor(encode("cat"))] )
T = torch.tensor([3])
print(model.forward(x, T))

x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
T = torch.tensor([3,3])
print(model.forward(x, T))

tensor([-9.0204], device='cuda:0')
tensor([-9.4975,    -inf], device='cuda:0')


When using the vowel <-> consonant HMM from above, notice that the forward algorithm returns $-\infty$ for $\mathbf{x} = ``\text{abb"}$. That's because our transition matrix says the probability of vowel -> vowel and consonant -> consonant is 0, so the probability of $``\text{abb"}$ happening is 0, and thus the log probability is $-\infty$.

#### *Side note: deriving the forward algorithm*

If you're interested in understanding how the forward algorithm actually computes $p(\mathbf{x})$, read this section; if not, skip to the next part on "Problem 2" (finding the most likely state sequence).



To derive the forward algorithm, start by deriving the forward variable:

$
\begin{align} 
    \alpha_{s,t} &= p(x_1, x_2, \dots, x_t, z_t=s) \\
     &= p(x_t | x_1, x_2, \dots, x_{t-1}, z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s)  \\ 
    &= p(x_t | z_t = s) \cdot p(x_1, x_2, \dots, x_{t-1}, z_t = s) \\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s', z_t = s) \right)\\
    &= p(x_t | z_t = s) \cdot \left( \sum_{s'} p(z_t = s | x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \cdot p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s') \right)\\
    &= \underbrace{p(x_t | z_t = s)}_{\text{emission model}} \cdot \left( \sum_{s'} \underbrace{p(z_t = s | z_{t-1}=s')}_{\text{transition model}} \cdot \underbrace{p(x_1, x_2, \dots, x_{t-1}, z_{t-1}=s')}_{\text{forward variable for previous timestep}} \right)\\
    &= b_s(x_t) \cdot \left( \sum_{s'} A_{s, s'} \cdot \alpha_{s',t-1} \right)
\end{align}
$

I'll explain how to get to each line of this equation from the previous line. 

Line 1 is the definition of the forward variable $\alpha_{s,t}$.

Line 2 is the chain rule ($p(A,B) = p(A|B) \cdot p(B)$, where $A$ is $x_t$ and $B$ is all the other variables).

In Line 3, we apply Assumption 2: the probability of observation $x_t$ depends only on the current state $z_t$.

In Line 4, we marginalize over all the possible states in the previous timestep $t-1$.

In Line 5, we apply the chain rule again.

In Line 6, we apply Assumption 1: the current state depends only on the previous state.

In Line 7, we substitute in the emission probability, the transition probability, and the forward variable for the previous timestep, to get the complete recursion.

The formula above can be used for $t = 2 \rightarrow T$. At $t=1$, there is no previous state, so instead of the transition matrix $A$, we use the state priors $\pi$, which tell us the probability of starting in each state. Thus for $t=1$, the forward variables are computed as follows:

$$\begin{align} 
\alpha_{s,1} &= p(x_1, z_1=s) \\
  &= p(x_1 | z_1 = s) \cdot p(z_1 = s)  \\ 
&= b_s(x_1) \cdot \pi_s
\end{align}$$

Finally, to compute $p(\mathbf{x}) = p(x_1, x_2, \dots, x_T)$, we marginalize over $\alpha_{s,T}$, the forward variables computed in the last timestep:

$$\begin{align*} 
p(\mathbf{x}) &= \sum_{s} p(x_1, x_2, \dots, x_T, z_T = s) \\ 
&= \sum_{s} \alpha_{s,T}
\end{align*}$$

You can get from this formulation to the log domain formulation by taking the log of the forward variable, and using these identities:
- $\text{log }(a \cdot b) = \text{log }a + \text{log }b$
- $\text{log }(a + b) = \text{log }(e^{\text{log }a} + e^{\text{log }b}) = \text{logsumexp}(\text{log }a, \text{log }b)$

### Problem 2-2. How do we compute $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x})$?

Given an observation sequence $\mathbf{x}$, we may want to find the most likely sequence of states that could have generated $\mathbf{x}$. (Given the sequence of selfies, we want to infer what cities the friend visited.) In other words, we want $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x})$.

We can use Bayes' rule to rewrite this expression:
$$\begin{align*} 
    \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{z}|\mathbf{x}) &= \underset{\mathbf{z}}{\text{argmax }} \frac{p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})}{p(\mathbf{x})} \\ 
    &= \underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})
\end{align*}$$

Hmm! That last expression, $\underset{\mathbf{z}}{\text{argmax }} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$, looks suspiciously similar to the intractable expression we encountered before introducing the forward algorithm, $\underset{\mathbf{z}}{\sum} p(\mathbf{x}|\mathbf{z}) p(\mathbf{z})$.

And indeed, just as the intractable *sum* over all $\mathbf{z}$ can be implemented efficiently using the forward algorithm, so too this intractable *argmax* can be implemented efficiently using a similar divide-and-conquer algorithm: the legendary Viterbi algorithm!

________

<u><b>The Viterbi Algorithm</b></u>

> for $s=1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,1} := b_s(x_1) \cdot \pi_s$\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,1} := 0$
>
> for $t = 2 \rightarrow T$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;for $s = 1 \rightarrow N$:\
> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\delta_{s,t} := b_s(x_t) \cdot \left( \underset{s'}{\text{max }} A_{s, s'} \cdot \delta_{s',t-1} \right)$\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$\psi_{s,t} := \underset{s'}{\text{argmax }} A_{s, s'} \cdot \delta_{s',t-1}$
> 
> $z_T^* := \underset{s}{\text{argmax }} \delta_{s,T}$\
> for $t = T-1 \rightarrow 1$:\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$z_{t}^* := \psi_{z_{t+1}^*,t+1}$
> 
> $\mathbf{z}^* := \{z_{1}^*, \dots, z_{T}^* \}$\
return $\mathbf{z}^*$
________

The Viterbi algorithm looks somewhat gnarlier than the forward algorithm, but it is essentially the same algorithm, with two tweaks: 1) instead of taking the sum over previous states, we take the max; and 2) we record the argmax of the previous states in a table, and loop back over this table at the end to get $\mathbf{z}^*$, the most likely state sequence. (And like the forward algorithm, we should run the Viterbi algorithm in the log domain for better numerical stability.) 

**TODO 3**: Let's add the Viterbi algorithm to our PyTorch model:

In [7]:
def viterbi(self, x, T):
  """
  x : IntTensor of shape (batch size, T_max)
  T : IntTensor of shape (batch size)
  Find argmax_z log p(x|z) for each (x) in the batch.
  """
  ##############################################################################
  #                          IMPLEMENT YOUR CODE                               #
  ##############################################################################
  
  psi = torch.zeros((x.shape[1], x.shape[0], self.N)) # (T_max, batch size, N)
  z_star = torch.zeros((x.shape[0], x.shape[1])) # (batch size, T_max)
  # initialization
  log_state_priors = torch.log(torch.nn.functional.softmax(self.unnormalized_state_priors, dim=0)) # (N)
  log_emission = self.emission_model.forward(x[:, 0]) # (batch size, N)
  delta = log_state_priors.reshape((1, -1)) + log_emission  # (batch size, N)

  # recursion
  for t in range(1, x.shape[1]):
    log_emission = self.emission_model.forward(x[:, t]) # (batch size, N)
    logsumexp, psi[t, :, :] = self.transition_model.maxmul(delta) # (batch size, N)
    delta = log_emission + logsumexp  # (batch size, N)

  # termination
  best_path_scores, z = torch.max(delta, dim=1) # (batch size) (batch size)
  
  # path backtracking
  z_star[:, x.shape[1]-1] = z # (batch size)
  for t in range(x.shape[1]-1, 0, -1):
    mask1 = [n for n in range(x.shape[0])]
    mask2 = [z[n] for n in range(x.shape[0])]
    z = psi[t, mask1, mask2]
    z_star[:, t-1] = z
  
  ##############################################################################
  #                          END OF YOUR CODE                                  #
  ##############################################################################
  return z_star, best_path_scores # return both the best path and its log probability

def transition_model_maxmul(self, log_alpha):
  # log_alpha (batch size, N)
  log_transition_matrix = torch.nn.functional.log_softmax(self.unnormalized_transition_matrix, dim=0) # (N, N)

  out1, out2 = maxmul(log_transition_matrix, log_alpha.transpose(0,1))
  return out1.transpose(0,1), out2.transpose(0,1)

def maxmul(log_A, log_B):
	"""
	log_A : m x n
	log_B : n x p
	output : m x p matrix

	Similar to the log domain matrix multiplication,
	this computes out_{i,j} = max_k log_A_{i,k} + log_B_{k,j}
	"""
	m = log_A.shape[0]
	n = log_A.shape[1]
	p = log_B.shape[1]

	log_A_expanded = torch.stack([log_A] * p, dim=2)
	log_B_expanded = torch.stack([log_B] * m, dim=0)

	elementwise_sum = log_A_expanded + log_B_expanded
	out1,out2 = torch.max(elementwise_sum, dim=1)

	return out1,out2

TransitionModel.maxmul = transition_model_maxmul
HMM.viterbi = viterbi

Try running Viterbi on an input sequence, given the vowel/consonant HMM:

In [8]:
x = torch.stack( [torch.tensor(encode("aba")), torch.tensor(encode("abb"))] )
T = torch.tensor([3,3])
print(model.viterbi(x, T))

(tensor([[1., 0., 1.],
        [1., 0., 0.]]), tensor([-9.4975,    -inf], device='cuda:0'))


For $\mathbf{x} = ``\text{aba"}$, the Viterbi algorithm returns $\mathbf{z}^* = \{1,0,1\}$. This corresponds to "vowel, consonant, vowel" according to the way we defined the states above, which is correct for this input sequence. Yay!

For $\mathbf{x} = ``\text{abb"}$, the Viterbi algorithm still returns a $\mathbf{z}^*$, but we know this is gibberish because "vowel, consonant, consonant" is impossible under this HMM, and indeed the log probability of this path is $-\infty$.

### Problem 2-3. Comparison: Forward vs. Viterbi

1. **TODO 4**: Compare the "forward score" (the log probability of all possible paths, returned by the forward algorithm) with the "Viterbi score" (the log probability of the maximum likelihood path, returned by the Viterbi algorithm). You can print out model's outputs. And please explain the result below.

In [9]:
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

print(model.forward(x, T))
print(model.viterbi(x, T)[1])

##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

tensor([-9.4975,    -inf], device='cuda:0')
tensor([-9.4975,    -inf], device='cuda:0')



2. **Explanation for TODO 4**:<br>

위의 결과에선 forward score과 Viterbi score을 비교한 결과가 같게 나왔다. forward score은 모델에서 observation sequence가 관측될 수 있는 확률을 반환하고 Viterbi score은 모델에서 임의의 state sequence가 observation sequence를 생성할 확률 중 최대값을 반환한다. 한편, 위의 input은 observation sequence를 생성할 수 있는 state sequence가 유일하여 forward path는 1개이고 이는 곧 Viterbi path와 동일하다.


### Problem 2-4. How do we train the model?





Earlier, we hard-coded an HMM to have certain behavior. What we would like to do instead is have the HMM learn to model the data on its own. And while it is possible to use supervised learning with an HMM (by hard-coding the emission model or the transition model) so that the states have a particular interpretation, the really cool thing about HMMs is that they are naturally unsupervised learners, so they can learn to use their different states to represent different patterns in the data, without the programmer needing to indicate what each state means.

Like many machine learning models, an HMM can be trained using maximum likelihood estimation, i.e.:

$$\theta^* = \underset{\theta}{\text{argmin }} -\sum_{\mathbf{x}^i}\text{log }p_{\theta}(\mathbf{x}^i)$$

where $\mathbf{x}^1, \mathbf{x}^2, \dots$ are training examples. 

The standard method for doing this is the Expectation-Maximization (EM) algorithm, which for HMMs is also called the "Baum-Welch" algorithm. In EM training, we alternate between an "E-step", where we estimate the values of the latent variables, and an "M-step", where the model parameters are updated given the estimated latent variables. (Think $k$-means, where you guess which cluster each data point belongs to, then reestimate where the clusters are, and repeat.) The EM algorithm has some nice properties: it is guaranteed at each step to decrease the loss function, and the E-step and M-step may have an exact closed form solution, in which case no pesky learning rates are required.

But because the HMM forward algorithm is differentiable with respect to all the model parameters, we can also just take advantage of automatic differentiation methods in libraries like PyTorch and try to minimize $-\text{log }p_{\theta}(\mathbf{x})$ directly, by backpropagating through the forward algorithm and running stochastic gradient descent. That means we don't need to write any additional HMM code to implement training: `loss.backward()` is all you need.

Here we will implement SGD training for an HMM in PyTorch. First, some helper classes:

In [11]:
import torch.utils.data
from collections import Counter
from sklearn.model_selection import train_test_split

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, lines):
    self.lines = lines # list of strings
    collate = Collate() # function for generating a minibatch from strings
    self.loader = torch.utils.data.DataLoader(self, batch_size=1024, num_workers=1, shuffle=True, collate_fn=collate)

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx):
    line = self.lines[idx].lstrip(" ").rstrip("\n").rstrip(" ").rstrip("\n")
    return line

class Collate:
  def __init__(self):
    pass

  def __call__(self, batch):
    """
    Returns a minibatch of strings, padded to have the same length.
    """
    x = []
    batch_size = len(batch)
    for index in range(batch_size):
      x_ = batch[index]

      # convert letters to integers
      x.append(encode(x_))

    # pad all sequences with 0 to have same length
    x_lengths = [len(x_) for x_ in x]
    T = max(x_lengths)
    for index in range(batch_size):
      x[index] += [0] * (T - len(x[index]))
      x[index] = torch.tensor(x[index])

    # stack into single tensor
    x = torch.stack(x)
    x_lengths = torch.tensor(x_lengths)
    return (x,x_lengths)

Let's load some training/testing data. By default, this will use the unix "words" file, but you could also use your own text file.

Execute the following 2 cells if you are using Google Colab, otherwise just skip. Rename the statement after `cd` with your current directory (e.g. `cd /content/drive/your/current/directory/`). Refer to Assignment1's data loading sceme if you have troubles.

In [12]:
# Execute this and below cell if you are using Google Colab, otherwise just skip.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [27]:
filename = "hmm_training.txt"

with open(filename, "r") as f:
  lines = f.readlines() # each line of lines will have one word

alphabet = list(Counter(("".join(lines))).keys())
train_lines, valid_lines = train_test_split(lines, test_size=0.1, random_state=42)
train_dataset = TextDataset(train_lines)
valid_dataset = TextDataset(valid_lines)

M = len(alphabet)

We will use a Trainer class for training and testing the model:



In [19]:
from tqdm import tqdm # for displaying progress bar

class Trainer:
  def __init__(self, model, lr):
    self.model = model
    self.lr = lr
    self.optimizer = torch.optim.Adam(model.parameters(), lr=self.lr, weight_decay=0.00001)
  
  def train(self, dataset):
    train_loss = 0
    num_samples = 0
    self.model.train()
    print_interval = 50
    for idx, batch in enumerate(tqdm(dataset.loader)):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      self.optimizer.zero_grad()
      loss.backward()
      self.optimizer.step()
      train_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        for _ in range(5):
          sampled_x, sampled_z = self.model.sample()
          print(decode(sampled_x))
          print(sampled_z)
    train_loss /= num_samples
    return train_loss

  def test(self, dataset):
    test_loss = 0
    num_samples = 0
    self.model.eval()
    print_interval = 50
    for idx, batch in enumerate(tqdm(dataset.loader)):
      x,T = batch
      batch_size = len(x)
      num_samples += batch_size
      log_probs = self.model(x,T)
      loss = -log_probs.mean()
      test_loss += loss.cpu().data.numpy().item() * batch_size
      if idx % print_interval == 0:
        print("loss:", loss.item())
        sampled_x, sampled_z = self.model.sample()
        print(decode(sampled_x))
        print(sampled_z)
    test_loss /= num_samples
    return test_loss

Finally, initialize the model and run the main training loop. Every 50 batches, the code will produce a few samples from the model. Over time, these samples should look more and more realistic.

In [20]:
# Initialize model
model = HMM(N=64, M=M)
print(M)
# Train the model
num_epochs = 10
trainer = Trainer(model, lr=0.01)

for epoch in range(num_epochs):
    print("========= Epoch %d of %d =========" % (epoch+1, num_epochs))
    train_loss = trainer.train(train_dataset)
    valid_loss = trainer.test(valid_dataset)

    print("========= Results: epoch %d of %d =========" % (epoch+1, num_epochs))
    print("train loss: %.2f| valid loss: %.2f\n" % (train_loss, valid_loss))

torch.save(model.state_dict(), 'HMM.pth')

54
========= Epoch 1 of 10 =========
whatisthis: HMM(
  (transition_model): TransitionModel()
  (emission_model): EmissionModel()
)


  0%|          | 0/208 [00:00<?, ?it/s]

loss: 80.51759338378906
pjBlnOuGzn
[57, 28, 48, 52, 20, 49, 14, 51, 15, 21]
zDpVcBtegX
[13, 29, 44, 14, 63, 19, 43, 45, 63, 56]
cbnewbkJIB
[13, 32, 34, 2, 20, 53, 33, 61, 56, 19]


  1%|▏         | 3/208 [00:00<00:19, 10.68it/s]

tgszRp-ODa
[53, 56, 38, 8, 24, 8, 55, 3, 31, 41]
zelJ-CQCeb
[13, 42, 21, 25, 17, 34, 52, 35, 35, 20]


...

loss: 24.62409210205078
rlaehfehhc
[48, 49, 8, 32, 30, 48, 8, 32, 30, 48]
tnacydcsnu
[48, 49, 8, 48, 8, 48, 48, 48, 49, 8]
ralalGxetu
[48, 8, 48, 8, 48, 48, 49, 8, 48, 8]
icknemotab
[8, 48, 3, 49, 8, 48, 8, 48, 8, 48]
rumorteMop
[48, 8, 48, 8, 32, 48, 8, 48, 8, 48]


 21%|██        | 5/24 [00:00<00:00, 26.52it/s]

loss: 24.747407913208008
cnesnediht
[48, 49, 8, 48, 49, 8, 48, 8, 32, 48]


100%|██████████| 24/24 [00:00<00:00, 40.51it/s]

========= Results: epoch 10 of 10 =========
train loss: 24.75| valid loss: 24.75



**TODO 5**: Try to decode some different sequences (e.g. proper words, misspelled words) with the Viterbi algorithm and give an explanation about the results.

In [65]:
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################

def get_viterbi_score_and_path(str):
  x = torch.stack( [torch.tensor(encode(str))] )
  T = torch.tensor([len(str)])
  print(str, model.viterbi(x,T)[1].cpu().data.numpy().item())

print("\n" ,"1. 영어 일상어")
get_viterbi_score_and_path("time")
get_viterbi_score_and_path("boy")
get_viterbi_score_and_path("girl")
get_viterbi_score_and_path("work")

print("\n" ,"2. 영어 전문어")
get_viterbi_score_and_path("benchmark")
get_viterbi_score_and_path("differentiated")
get_viterbi_score_and_path("laplace")
get_viterbi_score_and_path("fourier")

print("\n" ,"3. 프랑스 일상어")
get_viterbi_score_and_path("temps")
get_viterbi_score_and_path("Garcon")
get_viterbi_score_and_path("Fille")
get_viterbi_score_and_path("Une")

print("\n" ,"4. 영어 complex 오타 비교")
get_viterbi_score_and_path("complex")
get_viterbi_score_and_path("complecs")
get_viterbi_score_and_path("conplex")
get_viterbi_score_and_path("conplecs")


##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################


 1. 영어 일상어
time -9.742241859436035
boy -11.735644340515137
girl -11.534860610961914
work -13.94454574584961

 2. 영어 전문어
benchmark -30.31708526611328
differentiated -43.4920654296875
laplace -17.901363372802734
fourier -22.890518188476562

 3. 프랑스 일상어
temps -14.856160163879395
Garcon -18.604888916015625
Fille -18.491153717041016
Une -12.896363258361816

 4. 영어 complex 오타 비교
complex -22.877077102661133
complecs -23.162832260131836
conplex -23.08810043334961
conplecs -23.37385368347168


**Explanation for TODO 5:** <br>
write your answer here.

In [67]:
# 1. 일상적인 영어를 input으로 넣어주면 높은 확률의 viterbi path를 관찰할 수 있다. 
# 2. 영어 전문어를 input으로 넣어주면 낮은 확률의 viterbi path를 관찰할 수 있다. 
# 3. 프랑스 일상어를 input으로 넣어주면 영어의 일상어보단 훨씬 낮은 확률의 viterbi path를 확인 할 수 있다.
# 4. complex라는 영어의 단어를 가지고 여러 변형된 오타를 input으로 넣어주면 본래의 viterbi path보다 조금 낮은 score을 갖는 viterbi path를 확인 할 수 있다.
# 5. 1.과 3.의 차이를 활용해서 observation이 주어져 있을 때 어떤 언어인지 추론해보는데 응용될 수 있다.
# 6. 언어가 주어졌다고 가정하고, 4.를 응용해서 오타를 교정하는 알고리즘을 개발하는데 insight를 얻을 수 있다.